# Usage of `src.data.iem_query` module.

This notebook outlies the basic usage of the `src.data.iem_query` module. Used to query US weather stations data from the IEM database.
 
**Requirements**
 - A `.yml` file being a copy of `config/config_template.yml` contaning information about the flights to be downloaded, as well as a path to the credentials config file

**Helpful Links**

 - [Downloading and Interpreting IEM Data Wiki](https://github.com/Flight-Path-Analysis/FlightPathAnalysis/wiki/Downloading_and_Interpreting_IEM_Data)

 - [Iowa Environmental Mesonet](https://mesonet.agron.iastate.edu/)

## Basic Setup

In [1]:
import sys
# This variable should indicate the path from this Jupyter Notebook to the root directory of the repo.
root_path = '../'
# Adds the repo's root to the list of paths
sys.path.append(root_path)

# Package to read yml files
import yaml
# Package to handle file paths
import os
# Function to clear output from jupyter notebook
from IPython.display import clear_output
# Package for downloading IEM data 
from src.data import iem_query
# Utilities package
from src.common import utils

# Normalizing all paths to work on all operational systems
root_path = os.path.normpath(root_path) # Path from this notebook to the root directory
config_path_from_root = os.path.normpath('config/config_tutorial.yml') # Path from root to the desired config file
config_path = os.path.join(root_path, config_path_from_root) # Defining path from this notebook to config file

# Loading config file as a dictionary
with open(config_path, 'r', encoding='utf8') as file:
    config = yaml.safe_load(file)

# Defining "clear-output" function to feed into logger
def clear():
    clear_output(wait=True)

# Creates an instance of a logger class to log all that happens, optional (but encouraged).
logger = utils.Logger(config, clear_function=clear)

iem_querier = iem_query.Querier(config, logger=logger)


## Loading all stations and station properties

This code loads information for all weather stations in the US states. It returns the results in the form of a Pandas DataFrame.

It also, by default, cleans up the data to a more interpretable format

The columns of the dataframe are described below

  - **id**: [Identification Number] - The unique identifier assigned to each weather station.
  - **elevation**: [Elevation] - The height of the weather station above sea level, usually measured in meters or feet.
  - **sname**: [Station Name] - The official name of the weather station.
  - **time_domain**: [Time Domain] - The time zone or specific time reference in which the weather station operates or reports data.
  - **archive_begin**: [Archive Start Date] - The date when the weather station began archiving data.
  - **archive_end**: [Archive End Date] - The last date for which data is available in the archive (if the station is still operational, this might be ongoing).
  - **state**: [State] - The state or province where the weather station is located.
  - **country**: [Country] - The country where the weather station is situated.
  - **climate_site**: [Climate Site Identifier] - A unique code or identifier for the climate data site if it is part of a broader climate observation network.
  - **wfo**: [Weather Forecast Office] - The local National Weather Service office responsible for the area where the weather station is located.
  - **tzname**: [Time Zone Name] - The name of the time zone where the weather station is located, such as 'EST' for Eastern Standard Time.
  - **ncdc81**: [NCDC 1981-2010 Normals] - Reference to the National Climatic Data Center's climate normals for the period from 1981 to 2010.
  - **ncei91**: [NCEI 1991-2020 Normals] - Reference to the National Centers for Environmental Information's climate normals for the period from 1991 to 2020.
  - **ugc_county**: [UGC County Code] - The Universal Geographic Code for the county in which the weather station is located.
  - **ugc_zone**: [UGC Zone Code] - The Universal Geographic Code for the forecast zone in which the weather station is located.
  - **county**: [County] - The county in which the weather station is located.
  - **network**: [Network] - The network or system of weather stations to which this particular station belongs.
  - **lon**: [Longitude] - The longitudinal coordinate of the weather station's location, usually in decimal degrees.
  - **lat**: [Latitude] - The latitudinal coordinate of the weather station's location, usually in decimal degrees.
  - **sigma**: [Sigma] - A custom measure, indicating the width (in physical distance) of the influence of this weather station for interpolation purposes.



In [2]:
# Loads the information for all US weather stations.
# filters to keep only stations for which there's available data
# Dates are specified in config['base-configs']
stations = iem_querier.load_all_station_properties(filter_by_date=True)

stations

2023/11/09 11:45:50 : Computing stations sigma


,id,elevation,sname,time_domain,archive_begin,archive_end,state,country,climate_site,wfo,...,ncdc81,ncei91,ugc_county,ugc_zone,county,network,online,lon,lat,sigma
0,PADK,4.000000,ADAK NAS/MITCHELL,(1942-Now),1942-10-30,None,AK,US,AK2587,AFC,...,USW00025704,USW00025713,AKC016,AKZ191,Aleutians West,AK_ASOS,True,-176.6460,51.8780,585663.326384
1,PAKH,13.000000,AKHIOK,(2007-Now),2007-01-05,None,AK,US,AKTADQ,AFC,...,USW00025501,USW00025501,AKC164,AKZ171,Lake & Peninsula,AK_ASOS,True,-154.1800,56.9400,223526.931065
2,PAUT,20.684900,Akun,(2014-Now),2014-05-15,None,AK,US,AK2587,AFC,...,USC00502587,USC00502587,AKC016,AKZ185,Aleutians East,AK_ASOS,True,-165.6041,54.1446,208000.098725
3,PALP,7.998743,Alpine,(2002-Now),2002-01-19,None,AK,US,AK9869,AFG,...,USC00502126,USC00500235,AKC185,AKZ203,North Slope,AK_ASOS,True,-150.9447,70.3443,110303.682024
4,PAFM,88.000000,AMBLER,(1988-Now),1988-01-05,None,AK,US,AK3215,AFG,...,USC00503212,USC00503212,AKC188,AKZ217,Northwest Arctic,AK_ASOS,True,-157.8575,67.1063,114396.517958
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2539,HSG,1485.382800,Thermopolis,(2016-Now),2016-04-25,None,WY,US,WY8875,RIW,...,USC00488875,USC00488875,WYC017,WYZ282,Hot Springs,WY_ASOS,True,-108.3897,43.7136,68655.970839
2540,TOR,1282.000000,TORRINGTON MUNICIPAL AIRPORT,(1998-Now),1998-10-01,None,WY,US,WY5612,CYS,...,USW00094053,USW00094053,WYC015,WYZ310,Goshen,WY_ASOS,True,-104.1528,42.0647,59195.640734
2541,EAN,1453.103900,Wheatland,(2016-Now),2016-05-20,None,WY,US,WY9615,CYS,...,USC00489615,USC00489615,WYC031,WYZ067,Platte,WY_ASOS,True,-104.9286,42.0555,76300.298222
2542,WRL,1294.000000,WORLAND MUNICIPAL,(1972-Now),1972-12-31,None,WY,US,WY9785,RIW,...,USW00024062,USW00024062,WYC043,WYZ006,Fremont,WY_ASOS,True,-107.9508,43.9657,76241.449390


In [3]:
stations.columns

Index(['id', 'elevation', 'sname', 'time_domain', 'archive_begin',
       'archive_end', 'state', 'country', 'climate_site', 'wfo', 'tzname',
       'ncdc81', 'ncei91', 'ugc_county', 'ugc_zone', 'county', 'network',
       'online', 'lon', 'lat', 'sigma'],
      dtype='object')

## Loading data for a particular station in a date range

This code loads weather data for a specific weather station. It returns the results in the form of a Pandas DataFrame.

It also, by default, cleans up the data to a more interpretable format.

The columns of the dataframe are described below

  - **station:** The identifier for the weather station, usually an alphanumeric code.
  
  - **valid:** The timestamp indicating when the observation was taken.

  - **lon:** Longitude of the weather station.

  - **lat:** Latitude of the weather station.

  - **tmpf:** Temperature in degrees Fahrenheit.

  - **dwpf:** Dew point temperature in degrees Fahrenheit.

  - **relh:** Relative humidity, typically given as a percentage.

  - **drct:** Wind direction in degrees, with 0° or 360° representing north.

  - **sknt:** Wind speed in knots.

  - **p01i:** Precipitation for the past 1 hour, usually in inches.

  - **alti:** Altimeter setting, which is the current pressure at sea level.

  - **mslp:** Mean Sea Level Pressure.

  - **vsby:** Visibility, typically in statute miles.

  - **gust:** Wind gust speed in knots.

  - **skyc1, skyc2, skyc3, skyc4:** Sky cover for up to four levels. This can be in terms of clear (CLR), scattered (SCT), broken (BKN), overcast (OVC), etc.

  - **skyl1, skyl2, skyl3, skyl4:** Altitude (usually in feet) at which the respective sky cover (skyc1-4) levels are observed.

  - **wxcodes:** Weather codes representing specific weather phenomena, like rain (RA), snow (SN), etc.

  - **ice_accretion_1hr:** Amount of ice accretion (ice buildup) in the past 1 hour.

  - **ice_accretion_3hr:** Amount of ice accretion in the past 3 hours.

  - **ice_accretion_6hr:** Amount of ice accretion in the past 6 hours.

  - **peak_wind_gust:** Peak wind gust observed during the period.

  - **peak_wind_drct:** Direction from which the peak wind gust came.

  - **peak_wind_time:** Time when the peak wind gust was observed.

  - **feel:** "Feels like" temperature, which takes into account factors like wind chill or heat index.

  - **metar:** METAR is a format for reporting weather information, mainly used by pilots. This column would contain the raw METAR string for the observation.

  - **snowdepth:** Depth of snow on the ground, typically measured in inches.

In [6]:
# Loads weather data for the specific weather station fed as an argument
station_data = iem_querier.query_station_data(stations.iloc[0])
station_data

2023/11/07 10:41:19 : Requesting data for station PADK on base-url: http://mesonet.agron.iastate.edu/cgi-bin/request/asos.py?data=all&tz=Etc/UTC&format=comma&latlon=yes&year1=2022&month1=12&day1=31&year2=2023&month2=01&day2=01&


,timestamp,lon,lat,tmpf,relh,drct,sknt,p01i,skyc1,skyc2,...,sknt_E,sknt_N,smps_E,smps_N,tmpc,elevation,sigma,station,valid,metar
0,1672444800,-176.646,51.878,35.600000,93.080000,350.0,9.0,0.0,0.4,0.6,...,-1.562834e+00,8.863270,-8.039904e-01,4.559656,2.000000,4.0,585663.326384,PADK,2022-12-31 00:00,PADK 310000Z AUTO 35009KT SCT016 BKN023 BKN029...
1,1672445100,-176.646,51.878,35.600000,93.080000,350.0,8.0,0.0,0.4,0.4,...,-1.389185e+00,7.878462,-7.146581e-01,4.053028,2.000000,4.0,585663.326384,PADK,2022-12-31 00:05,PADK 310005Z AUTO 35008KT SCT013 SCT018 BKN025...
2,1672445400,-176.646,51.878,35.600000,93.080000,360.0,9.0,0.0,0.6,0.6,...,-2.204364e-15,9.000000,-1.134022e-15,4.629996,2.000000,4.0,585663.326384,PADK,2022-12-31 00:10,PADK 310010Z AUTO 36009KT BKN015 BKN020 BKN034...
3,1672445700,-176.646,51.878,35.600000,93.080000,360.0,9.0,0.0,0.6,0.6,...,-2.204364e-15,9.000000,-1.134022e-15,4.629996,2.000000,4.0,585663.326384,PADK,2022-12-31 00:15,PADK 310015Z AUTO 36009KT BKN015 BKN034 02/01 ...
4,1672446000,-176.646,51.878,35.600000,93.080000,350.0,9.0,0.0,0.2,0.4,...,-1.562834e+00,8.863270,-8.039904e-01,4.559656,2.000000,4.0,585663.326384,PADK,2022-12-31 00:20,PADK 310020Z AUTO 35009KT FEW012 SCT017 BKN034...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
325,1672530000,-176.646,51.878,35.869388,92.588367,360.0,5.0,0.0,0.2,0.6,...,-1.224647e-15,5.000000,-6.300122e-16,2.572220,2.149660,4.0,585663.326384,PADK,2022-12-31 23:40,PADK 312340Z AUTO 36005KT FEW011 BKN019 BKN025...
326,1672530300,-176.646,51.878,35.910204,92.513878,10.0,4.0,0.0,0.2,0.4,...,6.945927e-01,3.939231,3.573291e-01,2.026514,2.172336,4.0,585663.326384,PADK,2022-12-31 23:45,PADK 312345Z AUTO 01004KT FEW011 SCT016 BKN021...
327,1672530600,-176.646,51.878,35.951020,92.439388,360.0,7.0,0.0,0.2,0.6,...,-1.714506e-15,7.000000,-8.820171e-16,3.601108,2.195011,4.0,585663.326384,PADK,2022-12-31 23:50,PADK 312350Z AUTO 36007KT FEW011 BKN016 BKN021...
328,1672530900,-176.646,51.878,35.991837,92.364898,350.0,7.0,0.0,0.2,0.6,...,-1.215537e+00,6.893654,-6.253258e-01,3.546399,2.217687,4.0,585663.326384,PADK,2022-12-31 23:55,PADK 312355Z AUTO 35007KT FEW011 BKN016 BKN021...


In [7]:
# Loads weather data for all weather_stations | It will take a while
all_station_data = iem_querier.query_multiple_station_data(stations)
all_station_data

2023/11/07 10:53:22 : Requesting data for station P60 on base-url: http://mesonet.agron.iastate.edu/cgi-bin/request/asos.py?data=all&tz=Etc/UTC&format=comma&latlon=yes&year1=2022&month1=12&day1=31&year2=2023&month2=01&day2=01&


,timestamp,lon,lat,tmpf,relh,drct,sknt,p01i,skyc1,skyc2,...,sknt_E,sknt_N,smps_E,smps_N,tmpc,elevation,sigma,station,valid,metar
0,1672444800,-176.6460,51.8780,35.6,93.08,350.0,9.0,0.0,0.4,0.6,...,-1.562834e+00,8.863270,-8.039904e-01,4.559656,2.000000,4.0,585663.326384,PADK,2022-12-31 00:00,PADK 310000Z AUTO 35009KT SCT016 BKN023 BKN029...
1,1672445100,-176.6460,51.8780,35.6,93.08,350.0,8.0,0.0,0.4,0.4,...,-1.389185e+00,7.878462,-7.146581e-01,4.053028,2.000000,4.0,585663.326384,PADK,2022-12-31 00:05,PADK 310005Z AUTO 35008KT SCT013 SCT018 BKN025...
2,1672445400,-176.6460,51.8780,35.6,93.08,360.0,9.0,0.0,0.6,0.6,...,-2.204364e-15,9.000000,-1.134022e-15,4.629996,2.000000,4.0,585663.326384,PADK,2022-12-31 00:10,PADK 310010Z AUTO 36009KT BKN015 BKN020 BKN034...
3,1672445700,-176.6460,51.8780,35.6,93.08,360.0,9.0,0.0,0.6,0.6,...,-2.204364e-15,9.000000,-1.134022e-15,4.629996,2.000000,4.0,585663.326384,PADK,2022-12-31 00:15,PADK 310015Z AUTO 36009KT BKN015 BKN034 02/01 ...
4,1672446000,-176.6460,51.8780,35.6,93.08,350.0,9.0,0.0,0.2,0.4,...,-1.562834e+00,8.863270,-8.039904e-01,4.559656,2.000000,4.0,585663.326384,PADK,2022-12-31 00:20,PADK 310020Z AUTO 35009KT FEW012 SCT017 BKN034...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
416278,1672516560,-110.4211,44.5444,28.0,71.25,280.0,7.0,0.0,NaN,NaN,...,-6.893654e+00,1.215537,-3.546399e+00,0.625326,-2.222222,2368.0,100864.822210,P60,2022-12-31 19:56,KP60 311956Z AUTO 28007KT M02/M07 A2964 RMK AO...
416279,1672520160,-110.4211,44.5444,27.0,80.83,280.0,6.0,0.0,NaN,NaN,...,-5.908847e+00,1.041889,-3.039771e+00,0.535994,-2.777778,2368.0,100864.822210,P60,2022-12-31 20:56,KP60 312056Z AUTO 28006KT M03/M06 A2967 RMK AO...
416280,1672523760,-110.4211,44.5444,25.0,77.62,280.0,4.0,0.0,NaN,NaN,...,-5.416443e+00,0.955065,-2.786456e+00,0.491327,-3.888889,2368.0,100864.822210,P60,2022-12-31 21:56,KP60 312156Z AUTO VRB04KT M04/M07 A2968 RMK AO...
416281,1672527360,-110.4211,44.5444,23.0,80.85,280.0,5.0,0.0,NaN,NaN,...,-4.924039e+00,0.868241,-2.533142e+00,0.446661,-5.000000,2368.0,100864.822210,P60,2022-12-31 22:56,KP60 312256Z AUTO 28005KT M05/M08 A2970 RMK AO...
